In [20]:
import numpy as np
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import sys
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colorbar
from matplotlib import rc
from scipy.optimize import fsolve
from scipy.integrate import simpson
from numpy import trapz
from numpy.polynomial.polynomial import polyfit
from scipy import integrate
from matplotlib.pyplot import cm
from matplotlib.ticker import AutoMinorLocator

plt.rcParams["font.family"] = "Times New Roman"

mpl.rcParams['axes.linewidth'] = 2

In [21]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx,array[idx]

In [22]:
def k_point_grid_upper_half_bz(N):
    K_label = []
    Kx = []
    Ky = []
    x_span = int((int(N))/2)+1
    k_pair = 0
    for i in range(x_span):
        for j in range(x_span):
            if(j>=i):
              K_label.append(k_pair)
              Kx.append(i)
              Ky.append(j)
              k_pair = k_pair+1

              
    BZ = np.stack((K_label,Kx,Ky),axis = 1)
    
    return BZ,k_pair

In [23]:
def bz_cons(N,Graph_dir_LDOS):
   
   bz,k_pairs = k_point_grid_upper_half_bz(N)


   k_label_0 = bz[:,0]
   Kx_grid_0 = bz[:,1]
   Ky_grid_0 = bz[:,2]

   k_label_1 = []
   Kx_grid_1 = []
   Ky_grid_1 = []


   for ii in range(len(k_label_0)):

       if(Kx_grid_0[ii] != Ky_grid_0[ii]):
          kx = int(Ky_grid_0[ii])
          ky = int(Kx_grid_0[ii])
          Kx_grid_1.append(round(kx,1))
          Ky_grid_1.append(round(ky,1))
          k_label_1.append(k_label_0[ii])

   k_label_2 = np.append(k_label_0,np.array(k_label_1))
   Kx_grid_2 = np.append(Kx_grid_0,np.array(Kx_grid_1))
   Ky_grid_2 = np.append(Ky_grid_0,np.array(Ky_grid_1))
   
   k_label_3 = []
   Kx_grid_3 = []
   Ky_grid_3 = []
   
   k_label_4 = []
   Kx_grid_4 = []
   Ky_grid_4 = []

   k_label_5 = []
   Kx_grid_5 = []
   Ky_grid_5 = []
   
   
   for ii in range(len(k_label_2)):
       if Ky_grid_2[ii] > 0:
          kx_3 = round(np.cos(np.pi/2)*Kx_grid_2[ii]-np.sin(np.pi/2)*Ky_grid_2[ii],1)
          ky_3 = round(np.sin(np.pi/2)*Kx_grid_2[ii]+np.cos(np.pi/2)*Ky_grid_2[ii],1)
          k_label_3.append(k_label_2[ii])
          Kx_grid_3.append(kx_3)
          Ky_grid_3.append(ky_3)

       if Ky_grid_2[ii] > 0:
          kx_4 = round(np.cos(np.pi)*Kx_grid_2[ii]-np.sin(np.pi)*Ky_grid_2[ii],1)
          ky_4 = round(np.sin(np.pi)*Kx_grid_2[ii]+np.cos(np.pi)*Ky_grid_2[ii],1)
          k_label_4.append(k_label_2[ii])
          Kx_grid_4.append(kx_4)
          Ky_grid_4.append(ky_4)
  
       if Ky_grid_2[ii] > 0 and Kx_grid_2[ii] > 0:
          kx_5 = round(np.cos(1.5*np.pi)*Kx_grid_2[ii]-np.sin(1.5*np.pi)*Ky_grid_2[ii],1)
          ky_5 = round(np.sin(1.5*np.pi)*Kx_grid_2[ii]+np.cos(1.5*np.pi)*Ky_grid_2[ii],1)
          k_label_5.append(k_label_2[ii])
          Kx_grid_5.append(kx_5)
          Ky_grid_5.append(ky_5)
          
   k_label = np.append(k_label_2,np.append(np.array(k_label_3),np.append(np.array(k_label_4),np.array(k_label_5))))
   Kx_grid = np.append(Kx_grid_2,np.append(np.array(Kx_grid_3),np.append(np.array(Kx_grid_4),np.array(Kx_grid_5))))
   Ky_grid = np.append(Ky_grid_2,np.append(np.array(Ky_grid_3),np.append(np.array(Ky_grid_4),np.array(Ky_grid_5))))
   
   k_label_bz = []
   Kx_grid_bz = []
   Ky_grid_bz = []
   
   for i in range(len(k_label)):
       #print(Kx_grid,Ky_grid)
       if Kx_grid[i] > -1*int(int(N)/2) and Ky_grid[i] > -1*int(int(N)/2):
             k_label_bz.append(k_label[i])
             Kx_grid_bz.append(Kx_grid[i])
             Ky_grid_bz.append(Ky_grid[i])
          
       
   cm = matplotlib.colormaps.get_cmap('plasma')
   
   plt.figure(figsize = (20,20))
   ax = plt.gca()
   plt.xlabel('Kx',fontsize = 40)
   plt.ylabel('Ky',fontsize = 40)
   scat = plt.scatter(Kx_grid_bz,Ky_grid_bz,marker = "s",s = 5000)
   plt.xticks(fontsize = 40)
   plt.yticks(fontsize = 40)
   #divider = make_axes_locatable(ax)
   #cax = divider.append_axes("right", size="5%", pad=0.05)
   #cbar = plt.colorbar(scat, cax= cax)
   #cbar.ax.tick_params(labelsize = 30)
   for i, txt in enumerate(k_label_bz):
       ax.annotate(txt, (Kx_grid_bz[i], Ky_grid_bz[i]))
   
   plt.savefig('%s/Brillouin_zone_N_%s_scatter.png'%(Graph_dir_LDOS,N))
   print(len(k_label_bz),"k_list_length")
   
   return k_label_bz,Kx_grid_bz,Ky_grid_bz

In [24]:
def spectral_function_LDOS_plot_real_space_v2(Text_dir_ac_main,Graph_dir_LDOS,freq_no,sigma,N,U,Mu,Trot,dtau,omega_max,omega_min):
       
   Omega_gauss = np.zeros((len(U),freq_no,len(Trot)))
   LDOS_gauss = np.zeros((len(U),freq_no,len(Trot)))
   N_0_gauss = np.zeros((len(U),len(Trot)))
   LE_DOS_gauss = np.zeros((len(U),len(Trot)))
   Sum_rule = np.zeros((len(U),len(Trot)))
    
   start_ind = int(freq_no/2)-501
   end_ind = int(freq_no/2)+500 
   U_val =  np.zeros(len(U))
   for i in range(len(U)):
       U_val[i] = float(U[i])
       for k in range(len(Trot)):
               Text_dir_gf_r_avg_ac_main = "%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Spectral_functions_gaussian_sigma_%s_nfreq_%s"%(Text_dir_ac_main,N,U[i],dtau,Mu,dtau,Trot[k],sigma,freq_no)
               filename_spec_r = "%s/r_point_0/Spectral_function_real_space_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s_r_label_0_gaussian_sigma_%s_nfreq_%s_omega_max_%s_omega_min_%s.dat"%(Text_dir_gf_r_avg_ac_main,N,U[i],Mu,dtau,Trot[k],sigma,str(freq_no),omega_max,omega_min)
               Omega_gauss[i,:,k],LDOS_gauss[i,:,k] = np.loadtxt(filename_spec_r,unpack=True,usecols=[0,1])
               zero_ind,zero_val = find_nearest(Omega_gauss[i,:,k], 0.00000001)
               start_val = Omega_gauss[i,start_ind,k]
               end_val = Omega_gauss[i,end_ind,k]
               #print(start_val,end_val)
               LE_DOS_gauss[i][k] = (1/(end_val-start_val))*integrate.trapz(LDOS_gauss[i,start_ind:end_ind+1,k],Omega_gauss[i,start_ind:end_ind+1,k])
               N_0_gauss[i][k] = 0.5*(LDOS_gauss[i,zero_ind-1,k]+LDOS_gauss[i,zero_ind,k])
               Sum_rule[i,k] = integrate.trapz(LDOS_gauss[i,:,k],Omega_gauss[i,:,k])
           
   #Omega_mesh,U_mesh = np.meshgrid(Omega,U_val,indexing = 'xy')

   Graph_dir_LDOS_T_comp = "%s/LDOS_vs_T"%(Graph_dir_LDOS)
   if not os.path.exists(Graph_dir_LDOS_T_comp):
        os.makedirs(Graph_dir_LDOS_T_comp)

   Graph_dir_LDOS_U_comp = "%s/LDOS_vs_U"%(Graph_dir_LDOS)
   if not os.path.exists(Graph_dir_LDOS_U_comp):
        os.makedirs(Graph_dir_LDOS_U_comp)    
       
   T_val = np.zeros(len(Trot))
    
   for k in range(len(Trot)):
       beta = float(dtau)*float(Trot[k])
       T_val[k] = 1/beta

       plt.figure(figsize = (75,70))
       #plt.title(r"$N(\omega), N = %sx%s, \beta = %s, mu = %s$"%(N,N,str(round(beta,3)),Mu),fontsize = 40)
       ax = plt.gca()
       #plt.ylabel(r"$N(\omega)$",fontsize = 40)
       #plt.xlabel(r"$\omega$",fontsize = 40)
       plt.yticks([0.00, 0.05, 0.10, 0.15, 0.20],fontsize = 120)
       plt.xticks([-10.0,-5.0,0.0,5.0,10.0],fontsize = 120)
       plt.ylim(-0.005,0.22)
       plt.tick_params(axis='both', which='major', direction='out', pad=20,length = 30, width = 2.5,labelsize = 120)
       plt.tick_params(axis='both', which='minor', direction='out', pad=20,length = 15, width = 1.5,labelsize = 60)
       c1 = iter(cm.tab20(np.linspace(0, 1, len(U)+1))) #T_val = np.zeros(len(Trot))

       for i in range(len(U)):
           plt.plot(Omega_gauss[i,:,k],LDOS_gauss[i,:,k],color = next(c1),marker = "o",markersize = 10,label = "U=%s"%(U[i]))
       plt.legend(loc = 'best',ncol=3,fontsize = 75)
       plt.grid('True',which='both')
       plt.xlim(-12.0,12.0)
       plt.tight_layout()
       plt.savefig('%s/Gaussian_sigma_%s_LDOS_N_%s_mu_%s_dtau_%s_L_%s_omega_max_%s_omega_min_%s.png'%(Graph_dir_LDOS_U_comp,sigma,N,Mu,dtau,Trot[k],omega_max,omega_min))
       plt.close()

   for i in range(len(U)):
       #beta = float(dtau)*float(Trot[k])
       #T_val[k] = 1/beta

       plt.figure(figsize = (25,20))
       #plt.title(r"$N(\omega), N = %sx%s, U = %s, mu = %s$"%(N,N,U[i],Mu),fontsize = 40)
       ax = plt.gca()
       plt.xlim(0,4)
       #plt.ylim(0.06,0.20)
       #plt.ylabel(r"$N(\omega)$",fontsize = 40)
       #plt.xlabel(r"$\omega$",fontsize = 40)
       plt.yticks(fontsize = 100)
       plt.xticks(fontsize = 100)
       c1 = iter(cm.tab20(np.linspace(0, 1, len(Trot)+1))) #T_val = np.zeros(len(Trot))

       for k in range(len(Trot)-1,-1,-1):
           plt.plot(Omega_gauss[i,:,k],LDOS_gauss[i,:,k],color = next(c1),marker = "o",markersize = 5,label = "T=%s"%str(round(T_val[k],3)))
       plt.legend(loc = 'best',ncol=2,fontsize = 40)
       plt.grid('True',which='both')
       plt.tight_layout()
       #plt.yscale('log')
       plt.savefig('%s/Gaussian_sigma_%s_LDOS_N_%s_U_%s_mu_%s_dtau_%s_omega_max_%s_omega_min_%s.png'%(Graph_dir_LDOS_T_comp,sigma,N,U[i],Mu,dtau,omega_max,omega_min))
       plt.close()


   #===============================================================================================================================

    
   Omega_cut = str(round(end_val,4))
   plt.figure(figsize = (25,20))
   plt.xticks(fontsize = 80)
   plt.yticks(fontsize = 80)
   plt.title(r"LEDOS, $\Omega_c = %s$"%Omega_cut,fontsize = 60)    
   for i in range(len(U)):
        plt.plot(T_val,LE_DOS_gauss[i,:],marker = "o",markersize = 20, label = "U=%s"%U[i])
   plt.grid(True,which='both')
   plt.legend(loc = 'best',fontsize = 40)
   plt.savefig("%s/LEDOS_maxent_gaussian_sigma_%s_N_%s_dtau_%s_omega_cutoff_%s.png"%(Graph_dir_LDOS,sigma,N,dtau,Omega_cut))


   n_order = 4 
   Omega_cut = str(round(end_val,4))
   plt.figure(figsize = (25,20))
   plt.xticks(fontsize = 80)
   plt.yticks(fontsize = 80)
   plt.title(r"LEDOS, $\Omega_c = %s$"%Omega_cut,fontsize = 60)    
   T_fit = np.linspace(np.nanmin(T_val),np.nanmax(T_val),num = 101) 
   for i in range(0,len(U),4):
        coeff = np.polyfit(T_val,LE_DOS_gauss[i,:],n_order)
        polynomial = np.poly1d(coeff)
        plt.plot(T_val,LE_DOS_gauss[i,:],marker = "o",markersize = 20, label = "U=%s,QMC"%U[i])
        plt.plot(T_fit,polynomial(T_fit),linewidth=3, label = "U=%s,fit"%U[i])
   plt.grid(True,which='both')
   plt.legend(loc = 'best',fontsize = 40)
   plt.savefig("%s/LEDOS_inset_with_fit_maxent_gaussian_sigma_%s_N_%s_dtau_%s_omega_cutoff_%s_fit_order_%s.png"%(Graph_dir_LDOS,sigma,N,dtau,Omega_cut,str(n_order)))


    
   plt.figure(figsize = (25,20))
   plt.xticks(fontsize = 80)
   plt.yticks(fontsize = 80)
   plt.title(r"LEDOS, $\Omega_c = %s$"%Omega_cut,fontsize = 60)    
   for i in range(len(U)):
        plt.plot(T_val,N_0_gauss[i,:],marker = "o",markersize = 20, label = "U=%s"%U[i])
   plt.grid(True,which='both')
   plt.legend(loc = 'best',fontsize = 40)
   plt.savefig("%s/N0_maxent_gaussian_sigma_%s_N_%s_dtau_%s.png"%(Graph_dir_LDOS,sigma,N,dtau))

   figure = plt.figure()
   axes = figure.add_subplot(111)

   print(Sum_rule) 
# using the matshow() function 
   caxes = axes.matshow(Sum_rule, interpolation ='nearest')
   figure.colorbar(caxes)
 
   #axes.set_xticklabels(Trot)
   #axes.set_yticklabels(U)
   plt.savefig("%s/Sum_rule_gaussian_sigma_%s_N_%s_datu_%s.png"%(Graph_dir_LDOS,sigma,N,dtau))

   U_val_LDOS = np.asarray([3.20,3.30,3.40,3.50,3.60,3.70,3.80,3.90,4.00,4.10,4.20,4.300,4.40,4.50,4.700,4.800,4.900,5.00,5.100,5.200,5.400,5.500,6.000])
   T_c_LDOS =   np.asarray([0.30,0.30,0.30,0.30,0.30,0.30,0.30,0.33,0.33,0.33,0.33,0.345,0.40,0.40,0.417,0.417,0.455,0.50,0.526,0.526,0.588,0.714,1.000])

   #U_val_LDOS = np.asarray([3.20,3.30,3.40,3.50,3.60,3.70,3.80,3.90,4.00,4.10,4.20,4.300,4.40,4.50,4.700,4.800,4.900,5.00,5.100,5.200,5.400,5.500,6.000])
   #T_c_LDOS =   np.asarray([0.30,0.30,0.30,0.30,0.30,0.30,0.30,0.33,0.33,0.33,0.33,0.345,0.40,0.40,0.417,0.417,0.455,0.50,0.526,0.526,0.588,0.714,1.000])
    
   U_val_LDOS_lor = np.asarray([3.20,3.30,3.40,3.50,3.60,3.70,3.80,3.90,4.000,4.10,4.20,4.300,4.40,4.600,4.800,4.900,5.00,5.100,5.200])
   T_c_LDOS_lor =   np.asarray([0.30,0.30,0.30,0.30,0.30,0.30,0.30,0.33,0.345,0.33,0.33,0.345,0.40,0.435,0.435,0.455,0.50,0.556,0.556])


   U_val_TDOS = np.asarray([3.4247058823529413, 3.4835294117647058, 3.518823529411765, 3.5823529411764707, 3.695294117647059, 3.728235294117647, 
               3.7823529411764705, 3.881176470588235, 3.9752941176470586, 4.097647058823529, 4.198823529411765, 4.318823529411764]) 

   T_c_TDOS = np.asarray([0.47625899280575534,0.5,0.5258992805755395,0.556115107913669,0.587410071942446,0.6251798561151078,0.6672661870503598,
               0.7136690647482015,0.7687050359712231,0.833453237410072,0.9089928057553958,0.9996402877697843])

   U_ref= [2.24504054897068,4.2420461634435425,6.218714909544604,6.993791868083331] #, 7.9973008122101445] 
   Tc_ref=[0.10845290081097936,0.21101060511540903,0.5101060511540859,3.012170883125165] #, 8.020881898265154]

   z = np.polyfit(U_val_LDOS[8:], T_c_LDOS[8:], 2) 
   U_fit = np.linspace(4.1,6.0,num = 61)
   g_fit = np.poly1d(z) 
   plt.figure(figsize = (25,20))
   plt.xticks([3.5,4.0,4.5,5.0,5.5,6.0],fontsize = 120)
   plt.yticks([0.25,0.5,0.75,1.0],fontsize = 120)
   plt.xlim(3.4,6.1)
   plt.ylim(0.25,1.05) 
   plt.minorticks_on()
   plt.plot(U_val_LDOS,np.ma.masked_where(T_c_LDOS<0.32,T_c_LDOS), color = 'blue',marker = "s",markersize = 25,linestyle = 'None',label = r"$T_{c}^{\text{DOS}}$")
   plt.plot(U_val_TDOS[1:],T_c_TDOS[1:], color = 'red', marker = "^", markersize = 25, label = r"$T_{c}^{\text{TDOS}}$")
   plt.plot(U_fit,g_fit(U_fit), color = 'black',linewidth = 3) 
   plt.gca().xaxis.set_minor_locator(AutoMinorLocator())
   plt.gca().yaxis.set_minor_locator(AutoMinorLocator())

   plt.tick_params(axis='both', which='major', direction='out', pad=20,length = 30, width = 2.5,labelsize = 120)
   plt.tick_params(axis='both', which='minor', direction='out', pad=20,length = 15, width = 1.5,labelsize = 60)
    
   plt.grid(which='major', visible=True)  # Show grid lines only for major ticks
   plt.grid(which='minor', visible=False)
   #plt.grid(True, which = 'both')
   #plt.xlim(3.35,5.3) 
   plt.legend(loc = 'lower right', fontsize = 80)
   plt.tight_layout() 
   plt.savefig("%s/Gap_scales_N_%s_dtau_%s_gaussian_sigma_%s.png"%(Graph_dir_LDOS,N,dtau,sigma)) 


In [90]:
def spectral_function_LDOS_plot_real_space_scaling(Graph_dir_LDOS,freq_no,sigma,N1,N2,U,Mu,Trot,dtau,omega_max,omega_min):


   Text_dir_ac_main_n1 = "/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Text_files/Text_files_N_%s_analytic_continuation"%N1
   Text_dir_ac_main_n2 = "/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Text_files/Text_files_N_%s_analytic_continuation"%N2


    
   #Omega_gauss = np.zeros((len(U),freq_no,len(Trot)))
   #LDOS_gauss = np.zeros((len(U),freq_no,len(Trot)))
   #N_0_gauss = np.zeros((len(U),len(Trot)))
   #LE_DOS_gauss = np.zeros((len(U),len(Trot)))
   #Sum_rule = np.zeros((len(U),len(Trot)))
    
   #start_ind = int(freq_no/2)-501
   #end_ind = int(freq_no/2)+500 
   #U_val =  np.zeros(len(U))
   #for i in range(len(U)):
   #    U_val[i] = float(U[i])
   #    for k in range(len(Trot)):
   #            Text_dir_gf_r_avg_ac_main = "%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Spectral_functions_gaussian_sigma_%s_nfreq_%s"%(Text_dir_ac_main,N,U[i],dtau,Mu,dtau,Trot[k],sigma,freq_no)
   #            filename_spec_r = "%s/r_point_0/Spectral_function_real_space_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s_r_label_0_gaussian_sigma_%s_nfreq_%s_omega_max_%s_omega_min_%s.dat"%(Text_dir_gf_r_avg_ac_main,N,U[i],Mu,dtau,Trot[k],sigma,str(freq_no),omega_max,omega_min)
   #            Omega_gauss[i,:,k],LDOS_gauss[i,:,k] = np.loadtxt(filename_spec_r,unpack=True,usecols=[0,1])
   #            zero_ind,zero_val = find_nearest(Omega_gauss[i,:,k], 0.00000001)
   #            start_val = Omega_gauss[i,start_ind,k]
   #            end_val = Omega_gauss[i,end_ind,k]
               #print(start_val,end_val)
   #            LE_DOS_gauss[i][k] = (1/(end_val-start_val))*integrate.trapz(LDOS_gauss[i,start_ind:end_ind+1,k],Omega_gauss[i,start_ind:end_ind+1,k])
   #            N_0_gauss[i][k] = 0.5*(LDOS_gauss[i,zero_ind-1,k]+LDOS_gauss[i,zero_ind,k])
   #            Sum_rule[i,k] = integrate.trapz(LDOS_gauss[i,:,k],Omega_gauss[i,:,k])
           
   #Omega_mesh,U_mesh = np.meshgrid(Omega,U_val,indexing = 'xy')

   Graph_dir_LDOS_T_comp = "%s/LDOS_vs_T"%(Graph_dir_LDOS)
   if not os.path.exists(Graph_dir_LDOS_T_comp):
        os.makedirs(Graph_dir_LDOS_T_comp)

   Graph_dir_LDOS_U_comp = "%s/LDOS_vs_U"%(Graph_dir_LDOS)
   if not os.path.exists(Graph_dir_LDOS_U_comp):
        os.makedirs(Graph_dir_LDOS_U_comp)    
       
   T_val = np.zeros(len(Trot))
    
   for k in range(len(Trot)):
       beta = float(dtau)*float(Trot[k])
       T_val[k] = 1/beta

       plt.figure(figsize = (25,20))
       ax = plt.gca()
       plt.yticks([0.00, 0.05, 0.10, 0.15, 0.20],fontsize = 120)
       plt.xticks([-10.0,-5.0,0.0,5.0,10.0],fontsize = 120)
       plt.ylim(-0.005,0.175)
       plt.tick_params(axis='both', which='major', direction='out', pad=20,length = 30, width = 2.5,labelsize = 120)
       plt.tick_params(axis='both', which='minor', direction='out', pad=20,length = 15, width = 1.5,labelsize = 60)
       #c1 = iter(cm.tab20(np.linspace(0, 1, len(U)+1))) #T_val = np.zeros(len(Trot))

       for i in range(len(U)):
           #col = next(c1)
           Text_dir_gf_r_avg_ac_main_n1 = "%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Spectral_functions_gaussian_sigma_%s_nfreq_%s"%(Text_dir_ac_main_n1,N1,U[i],dtau,Mu,dtau,Trot[k],sigma,freq_no)
           filename_spec_r_n1 = "%s/r_point_0/Spectral_function_real_space_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s_r_label_0_gaussian_sigma_%s_nfreq_%s_omega_max_%s_omega_min_%s.dat"%(Text_dir_gf_r_avg_ac_main_n1,N1,U[i],Mu,dtau,Trot[k],sigma,str(freq_no),omega_max,omega_min)
           omega_gauss_n1,LDOS_gauss_n1 = np.loadtxt(filename_spec_r_n1,unpack=True,usecols=[0,1])           
           
           Text_dir_gf_r_avg_ac_main_n2 = "%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Spectral_functions_gaussian_sigma_%s_nfreq_%s"%(Text_dir_ac_main_n2,N2,U[i],dtau,Mu,dtau,Trot[k],sigma,freq_no)
           filename_spec_r_n2 = "%s/r_point_0/Spectral_function_real_space_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s_r_label_0_gaussian_sigma_%s_nfreq_%s_omega_max_%s_omega_min_%s.dat"%(Text_dir_gf_r_avg_ac_main_n2,N2,U[i],Mu,dtau,Trot[k],sigma,str(freq_no),omega_max,omega_min)
           omega_gauss_n2,LDOS_gauss_n2 = np.loadtxt(filename_spec_r_n2,unpack=True,usecols=[0,1])           

           if U[i] == '3.50':
              plt.plot(omega_gauss_n1,LDOS_gauss_n1,color = 'blue', linestyle = 'dashed',linewidth = 8,label = 'N = 10x10')
              plt.plot(omega_gauss_n2,LDOS_gauss_n2,color = 'blue', linestyle = 'solid',linewidth = 8, label = 'N = 16x16') #,fillstyle='none') #,label = "U=%s"%(U[i]))           

           if U[i] == '5.50':
              plt.plot(omega_gauss_n1,LDOS_gauss_n1,color = 'brown', linestyle = 'dashed',linewidth = 8) #,label = "U=%s"%(U[i]))
              plt.plot(omega_gauss_n2,LDOS_gauss_n2,color = 'brown', linestyle = 'solid',linewidth = 8) #,fillstyle='none') #,label = "U=%s"%(U[i]))           

           if U[i] == '8.00':
              plt.plot(omega_gauss_n1,LDOS_gauss_n1,color = 'olive', linestyle = 'dashed',linewidth = 8) #, label = "U=%s"%(U[i]))
              plt.plot(omega_gauss_n2,LDOS_gauss_n2,color = 'olive', linestyle = 'solid',linewidth = 8) #,fillstyle='none') #,label = "U=%s"%(U[i]))           
             
       plt.legend(loc = 'best',ncol=1,fontsize = 75)     
       plt.grid('True',which='both')
       plt.xlim(-12.0,12.0)
       plt.tight_layout()
       plt.savefig('%s/Gaussian_sigma_%s_LDOS_mu_%s_dtau_%s_L_%s_omega_max_%s_omega_min_%s_scaling.png'%(Graph_dir_LDOS_U_comp,sigma,Mu,dtau,Trot[k],omega_max,omega_min))
       plt.close()

       #================================ Plotting stuff for 10x10 ============================================

       plt.figure(figsize = (25,20))
       ax = plt.gca()
       plt.yticks([0.00, 0.05, 0.10, 0.15, 0.20],fontsize = 120)
       plt.xticks([-10.0,-5.0,0.0,5.0,10.0],fontsize = 120)
       plt.ylim(-0.005,0.175)
       plt.tick_params(axis='both', which='major', direction='out', pad=20,length = 30, width = 2.5,labelsize = 120)
       plt.tick_params(axis='both', which='minor', direction='out', pad=20,length = 15, width = 1.5,labelsize = 60)
       c1 = ['blue', 'orange', 'red','brown', 'darkgoldenrod','olive'] #iter(cm.tab20(np.linspace(0, 1, len(U)+1))) #T_val = np.zeros(len(Trot))

       for i in range(len(U)):
           col = c1[i]
           Text_dir_gf_r_avg_ac_main_n1 = "%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Spectral_functions_gaussian_sigma_%s_nfreq_%s"%(Text_dir_ac_main_n1,N1,U[i],dtau,Mu,dtau,Trot[k],sigma,freq_no)
           filename_spec_r_n1 = "%s/r_point_0/Spectral_function_real_space_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s_r_label_0_gaussian_sigma_%s_nfreq_%s_omega_max_%s_omega_min_%s.dat"%(Text_dir_gf_r_avg_ac_main_n1,N1,U[i],Mu,dtau,Trot[k],sigma,str(freq_no),omega_max,omega_min)
           omega_gauss_n1,LDOS_gauss_n1 = np.loadtxt(filename_spec_r_n1,unpack=True,usecols=[0,1])              
           plt.plot(omega_gauss_n1,LDOS_gauss_n1,color = col, linestyle = 'dashed',linewidth = 8,label = "U=%s"%(U[i]))
             
       #plt.legend(loc = 'best',ncol=6,fontsize = 75)     
       plt.grid('True',which='both')
       plt.xlim(-12.0,12.0)
       plt.tight_layout()
       plt.savefig('%s/Gaussian_sigma_%s_LDOS_mu_%s_dtau_%s_L_%s_omega_max_%s_omega_min_%s_N_%s.png'%(Graph_dir_LDOS_U_comp,sigma,Mu,dtau,Trot[k],omega_max,omega_min,N1))
       plt.close()

       #===================================== Plotting stuff for 16x16 ================================== 

       plt.figure(figsize = (25,20))
       ax = plt.gca()
       plt.yticks([0.00, 0.05, 0.10, 0.15, 0.20],fontsize = 120)
       plt.xticks([-10.0,-5.0,0.0,5.0,10.0],fontsize = 120)
       plt.ylim(-0.005,0.175)
       plt.tick_params(axis='both', which='major', direction='out', pad=20,length = 30, width = 2.5,labelsize = 120)
       plt.tick_params(axis='both', which='minor', direction='out', pad=20,length = 15, width = 1.5,labelsize = 60)
       #c1 = iter(cm.tab20(np.linspace(0, 1, len(U)+1))) #T_val = np.zeros(len(Trot))

       for i in range(len(U)):
           col = c1[i]
           Text_dir_gf_r_avg_ac_main_n2 = "%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Spectral_functions_gaussian_sigma_%s_nfreq_%s"%(Text_dir_ac_main_n2,N2,U[i],dtau,Mu,dtau,Trot[k],sigma,freq_no)
           filename_spec_r_n2 = "%s/r_point_0/Spectral_function_real_space_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s_r_label_0_gaussian_sigma_%s_nfreq_%s_omega_max_%s_omega_min_%s.dat"%(Text_dir_gf_r_avg_ac_main_n2,N2,U[i],Mu,dtau,Trot[k],sigma,str(freq_no),omega_max,omega_min)
           omega_gauss_n2,LDOS_gauss_n2 = np.loadtxt(filename_spec_r_n2,unpack=True,usecols=[0,1])           
           plt.plot(omega_gauss_n2,LDOS_gauss_n2,color = col, linestyle = 'solid',linewidth = 8) #,fillstyle='none') #,label = "U=%s"%(U[i]))           
       #plt.legend(loc = 'best',ncol=1,fontsize = 75)     
       plt.grid('True',which='both')
       plt.xlim(-12.0,12.0)
       plt.tight_layout()
       plt.savefig('%s/Gaussian_sigma_%s_LDOS_mu_%s_dtau_%s_L_%s_omega_max_%s_omega_min_%s_N_%s.png'%(Graph_dir_LDOS_U_comp,sigma,Mu,dtau,Trot[k],omega_max,omega_min,N2))
       plt.close()

In [91]:
def spectral_function_LDOS_plot_momentum_space_v2(Text_dir_ac_main,Graph_dir_LDOS,freq_no,sigma,N,U,Mu,Trot,dtau,omega_max,omega_min):


   klab,kx_grid,ky_grid = bz_cons(N,Graph_dir_LDOS)
   Omega_gaus = np.zeros((len(U),freq_no,len(Trot)))
   LDOS_gaus = np.zeros((len(U),freq_no,len(Trot)))
   #klab = bz[:,0]
   #print(klab) 
   #start_ind = int(freq_no/2)-501
   #end_ind = int(freq_no/2)+500 
   U_val =  np.zeros(len(U))
   for i in range(len(U)):
       U_val[i] = float(U[i])
       for k in range(len(Trot)):
           
           Text_dir_gf_k_avg_ac_main = "%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Spectral_functions_gaussian_sigma_%s_nfreq_%s"%(Text_dir_ac_main,N,U[i],dtau,Mu,dtau,Trot[k],sigma,freq_no)
           for kk in range(len(klab)):
               k_point = str(klab[kk])
               filename_spec_k = "%s/k_point_%s/Spectral_function_momentum_space_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s_k_label_%s_gaussian_sigma_%s_nfreq_%s_omega_max_%s_omega_min_%s.dat"%(Text_dir_gf_k_avg_ac_main,k_point,N,U[i],Mu,dtau,Trot[k],k_point,sigma,str(freq_no),omega_max,omega_min)
               omega_gaus,a_k_gaus = np.loadtxt(filename_spec_k,unpack=True,usecols=[0,1])
               x_a_k_gaus = np.add(LDOS_gaus[i,:,k],a_k_gaus)
               LDOS_gaus[i,:,k] = np.copy(x_a_k_gaus)
           Omega_gaus[i,:,k] = np.copy(omega_gaus)    
           

   Graph_dir_LDOS_T_comp = "%s/LDOS_vs_T"%(Graph_dir_LDOS)
   if not os.path.exists(Graph_dir_LDOS_T_comp):
        os.makedirs(Graph_dir_LDOS_T_comp)

   Graph_dir_LDOS_U_comp = "%s/LDOS_vs_U"%(Graph_dir_LDOS)
   if not os.path.exists(Graph_dir_LDOS_U_comp):
        os.makedirs(Graph_dir_LDOS_U_comp)    
       
   T_val = np.zeros(len(Trot))
    
   for k in range(len(Trot)):
       beta = float(dtau)*float(Trot[k])
       T_val[k] = 1/beta

       plt.figure(figsize = (25,20))
       #plt.title(r"$N(\omega), N = %sx%s, \beta = %s, mu = %s$"%(N,N,str(round(beta,3)),Mu),fontsize = 40)
       ax = plt.gca()
       #plt.ylabel(r"$N(\omega)$",fontsize = 40)
       #plt.xlabel(r"$\omega$",fontsize = 40)
       plt.yticks(fontsize = 80)
       plt.xticks(fontsize = 80)
       c1 = iter(cm.tab20(np.linspace(0, 1, len(U)+1))) #T_val = np.zeros(len(Trot))

       for i in range(len(U)):
           plt.plot(Omega_gaus[i,:,k],(1/len(klab))*LDOS_gaus[i,:,k],color = next(c1),marker = "o",markersize = 10,label = "U=%s"%(U[i]))
       plt.legend(loc = 'best',ncol=4,fontsize = 70)
       plt.grid('True',which='both')
       plt.xlim(-10.0,10.0)
       plt.tight_layout()
       plt.savefig('%s/Gaussian_sigma_%s_LDOS_N_%s_mu_%s_dtau_%s_L_%s_omega_max_%s_omega_min_%s.png'%(Graph_dir_LDOS_U_comp,sigma,N,Mu,dtau,Trot[k],omega_max,omega_min))
       plt.close()

   for i in range(len(U)):
       #beta = float(dtau)*float(Trot[k])
       #T_val[k] = 1/beta

       plt.figure(figsize = (25,20))
       #plt.title(r"$N(\omega), N = %sx%s, U = %s, mu = %s$"%(N,N,U[i],Mu),fontsize = 40)
       ax = plt.gca()
       plt.xlim(0,2)
       plt.ylim(0.05,0.2)
       #plt.ylim(0.06,0.20)
       #plt.ylabel(r"$N(\omega)$",fontsize = 40)
       #plt.xlabel(r"$\omega$",fontsize = 40)
       plt.yticks(fontsize = 80)
       plt.xticks(fontsize = 80)
       c1 = iter(cm.tab20(np.linspace(0, 1, len(Trot)+1))) #T_val = np.zeros(len(Trot))

       for k in range(len(Trot)-1,-1,-1):
           plt.plot(Omega_gaus[i,:,k],(1/len(klab))*LDOS_gaus[i,:,k],color = next(c1),marker = "o",markersize = 5,label = "T=%s"%str(round(T_val[k],3)))
       plt.yscale('log')
       plt.legend(loc = 'best',ncol=4,fontsize = 40)
       plt.grid('True',which='both')
       plt.tight_layout()
       #plt.yscale('log')
       plt.savefig('%s/Gaussian_sigma_%s_LDOS_N_%s_U_%s_mu_%s_dtau_%s_omega_max_%s_omega_min_%s.png'%(Graph_dir_LDOS_T_comp,sigma,N,U[i],Mu,dtau,omega_max,omega_min))
       plt.close()
  

In [92]:
def main():

    N1 = "10"
    N2 = "16"
    Dtau = "0.05"
    Omega_max = "25.0"
    Omega_min = "-25.0"
    Freq_no = 2501
    
    U = ["3.50","4.00","5.00","5.50","6.00","8.00"] #4.80","4.90","5.00","5.10","5.20","5.30","5.40","5.50","5.60","5.70","5.75"] #,"4.10","4.20","4.30","4.40","4.50","4.60","4.70","4.80","4.90","5.00"] #,"5.10","5.50","6.00"] #,"8.00"] #["3.30","3.40","3.50","3.60","3.70","3.80","3.90","4.00","4.10","4.20","4.30","4.40","4.50","4.60","4.70","4.80","4.90","5.00","5.10"]
    Mu = "0.00"
    Trot = ["30"] #["20","22","24","26","28","30","32","34","36","38","40","42","44","46","48","50"] #,"22","24","26","28","30","32"] #,"30","32","34","36","38","40"] #,"34","36","38","40"] #,"42","44","46","48","50"] #,"52","54","56","60"] #,"32","34","36","38","40","42","44","46","48","50","52","54","56","60"]
    Sigma = ["2.0"] #,"2.0"] #,"2.0"]


    #Text_dir_ac_main = "/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Text_files/Text_files_N_%s_analytic_continuation"%N
       
    for h in range(len(Sigma)):
        Graph_dir_ac_LDOS = "/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Graphs/Graphs_N_%s_maxent/Graphs_N_%s_dtau_%s_maxent_LDOS_half_filling_plots/Density_of_states_gaussian_default_model_sigma_%s_omega_max_%s_omega_min_%s"%(N1,N1,Dtau,Sigma[h],Omega_max,Omega_min)
        if not os.path.exists(Graph_dir_ac_LDOS):
              os.makedirs(Graph_dir_ac_LDOS)

        #Graph_dir_ac_k_LDOS = "/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Graphs/Graphs_N_%s_maxent/Graphs_N_%s_dtau_%s_maxent_LDOS_half_filling_plots_momentum_space/Density_of_states_gaussian_default_model_sigma_%s_omega_max_%s_omega_min_%s"%(N,N,Dtau,Sigma[h],Omega_max,Omega_min)
        #if not os.path.exists(Graph_dir_ac_k_LDOS):
        #      os.makedirs(Graph_dir_ac_k_LDOS)
            
        #Klab, Kx_cord,Ky_cord  = bz_cons(N,Graph_dir_ac_k_LDOS)
        #spectral_function_LDOS_plot_momentum_space_v2(Text_dir_ac_main,Graph_dir_ac_k_LDOS,Freq_no,Sigma[h],N,U,Mu,Trot,Dtau,Omega_max,Omega_min)
        #spectral_function_LDOS_plot_real_space_v2(Text_dir_ac_main,Graph_dir_ac_LDOS,Freq_no,Sigma[h],N,U,Mu,Trot,Dtau,Omega_max,Omega_min)
        spectral_function_LDOS_plot_real_space_scaling(Graph_dir_ac_LDOS,Freq_no,Sigma[h],N1,N2,U,Mu,Trot,Dtau,Omega_max,Omega_min)
                   
        #spectral_function_LDOS_plot_v2(Text_dir_ac_main,Graph_dir_ac_LDOS,Freq_no,Sigma[h],N,U,Mu,Trot,Dtau,Omega_max,Omega_min,Klab)
        #spectral_function_LDOS_gap(Text_dir_ac_main,Graph_dir_ac_LDOS,Freq_no,Sigma[h],N,U,Mu,Trot,Dtau,Omega_max,Omega_min,Klab)

In [93]:
main()

In [81]:
plt.

SyntaxError: invalid syntax (457645448.py, line 1)

In [ ]:
U_val_LDOS = np.asarray([3.20,3.30,3.40,3.50,3.60,3.70,3.80,3.90,4.00,4.10,4.20,4.300,4.40,4.50,4.700,4.800,4.900,5.00,5.100,5.200,5.500,6.000]) #,6.500,7.000,7.500,8.000])
T_c_LDOS =   np.asarray([0.30,0.30,0.30,0.30,0.30,0.30,0.30,0.33,0.33,0.33,0.33,0.345,0.40,0.40,0.417,0.417,0.455,0.50,0.526,0.556,0.667,0.714]) #,0.769,0.833,0.909,1.000])

U_val_LDOS_lor = np.asarray([3.20,3.30,3.40,3.50,3.60,3.70,3.80,3.90,4.000,4.10,4.20,4.300,4.40,4.600,4.800,4.900,5.00,5.100,5.200])
T_c_LDOS_lor =   np.asarray([0.30,0.30,0.30,0.30,0.30,0.30,0.30,0.33,0.345,0.33,0.33,0.345,0.40,0.435,0.435,0.455,0.50,0.556,0.556])


U_val_TDOS = np.asarray([3.4247058823529413, 3.4835294117647058, 3.518823529411765, 3.5823529411764707, 3.695294117647059, 3.728235294117647, 
               3.7823529411764705, 3.881176470588235, 3.9752941176470586, 4.097647058823529, 4.198823529411765, 4.318823529411764]) 

T_c_TDOS = np.asarray([0.47625899280575534,0.5,0.5258992805755395,0.556115107913669,0.587410071942446,0.6251798561151078,0.6672661870503598,
               0.7136690647482015,0.7687050359712231,0.833453237410072,0.9089928057553958,0.9996402877697843])

N = "10"
Dtau = "0.05"
Omega_max = "25.0"
Omega_min = "-25.0"
Freq_no = 2501
Sigma = "2.0"
dtau = "0.05"
Graph_dir_LDOS = "/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Graphs/Graphs_N_%s_maxent/Graphs_N_%s_dtau_%s_maxent_LDOS_half_filling_plots/Density_of_states_gaussian_default_model_sigma_%s_omega_max_%s_omega_min_%s"%(N,N,Dtau,Sigma,Omega_max,Omega_min)

            
z = np.polyfit(U_val_LDOS[7:], T_c_LDOS[7:], 2) 
U_fit = np.linspace(3.9,6.0,num = 81)
g_fit = np.poly1d(z) 
plt.figure(figsize = (25,20))
plt.xticks([3.0,3.5,4.0,4.5,5.0,5.5,6.0],fontsize = 120)
plt.yticks(fontsize = 120)
plt.plot(U_val_LDOS,np.ma.masked_where(T_c_LDOS<0.33,T_c_LDOS), color = 'blue',marker = "s",markersize = 25,linestyle = 'None',label = r"$T^{*}$, LDOS")
plt.plot(U_val_TDOS,T_c_TDOS, color = 'red', marker = "^", markersize = 25, label = r"$T^{*}$, TDOS")
plt.plot(U_fit,g_fit(U_fit), color = 'black',linewidth = 3) 
plt.grid(True, which = 'both')
   #plt.xlim(3.35,5.3) 
plt.legend(loc = 'best', fontsize = 60)
plt.tight_layout() 
plt.savefig("%s/Gap_scales_N_%s_dtau_%s_gaussian_sigma_%s.png"%(Graph_dir_LDOS,N,dtau,Sigma)) 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import LogLocator, NullFormatter

# Sample data
x = np.linspace(1, 10, 100)
y = np.log(x)

# Create a log plot
plt.plot(x, y)

# Set log scale for the x-axis
plt.xscale('log')

# Turn off minor ticks
plt.minorticks_off()
plt.grid(which='both')
plt.gca().xaxis.set_major_locator(LogLocator(base=10.0, numticks=20))
plt.gca().xaxis.set_minor_locator(LogLocator(base=10.0, subs='auto', numticks=10))

# Optionally, turn off minor tick labels
plt.gca().xaxis.set_minor_formatter(NullFormatter())

# Show the plot
plt.show()
# Show the plot
plt.show()